# 2. Aligning ACS and EPA Data

Our historical air quality data from the EPA is provided station by station, and the positions of each station is provided by coordinates. To make useful comparisons, we want to align the data - decide for each year which exact air quality data points align with each CBSA GeoID/region in ACS.

We'll refer to these as `epa_point` and `acs_region` respectively.

In [1]:
# set up path to app credentials - see exploration/README.md
%env GOOGLE_APPLICATION_CREDENTIALS=../google_app_credentials.json

# set up bigquery client
from google.cloud import bigquery
bq = bigquery.Client()

# set up some parameters
BQ_DATASET_ID = 'data'

env: GOOGLE_APPLICATION_CREDENTIALS=../google_app_credentials.json


In [2]:
# set up some dependencies
import geopandas as gp
import json

First we'll load up the relevant data, which should already be set up.

In [4]:
# set up acs utils
resp = bq.query('''
    SELECT DISTINCT do_date AS year
    FROM `eosc410-project.data.acs_cbsa_20*`
    ORDER BY do_date ASC
''')
acs_years = [row["year"] for row in resp]

def load_geojson(y) -> gp.GeoDataFrame:
    print('=> loading %s' % y)
    geo = gp.read_file('../_data/tmp/acs_cbsa_%s/geojson.json' % y)
    return geo

def get_acs_year(i: int) -> str:
    return str(int(acs_years[0]) + i)

In [5]:
# look at what our geojson looks like
test_year = get_acs_year(0)
test_region_geojson = load_geojson(test_year)
print('%s has %d features (regions)' % (test_year, len(test_region_geojson)))

test_region = test_region_geojson.loc[0] # first region in this year
test_region_id = test_region['CBSAFP']   # current metropolitan statistical area/micropolitan statistical area code
                                         # see https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2014/TGRSHP2014_TechDoc_Ch3.pdf
test_region_name = test_region['NAME']   # this region has a name
test_region_geo = test_region.geometry   # this region has a geometry
print('%s: %s' % (test_region_id, test_region_name))
print(json.dumps(test_region_geo.__geo_interface__)[:100] + "...")

=> loading 2007
2007 has 952 features (regions)
27980: Kahului-Wailuku, HI
{"type": "MultiPolygon", "coordinates": [[[[-156.70540599999998, 20.82632], [-156.711227, 20.832228]...


In [6]:
# for each year and acs region, we want to get associated station numbers
# try with the test region: find all EPA points that fall within this region's geometry in that year
resp = bq.query('''
SELECT DISTINCT site_num, address, cbsa_name
FROM `eosc410-project.data.epa_air_quality_annual` as epa
WHERE
  ST_WITHIN(ST_GEOGPOINT(epa.longitude, epa.latitude), ST_GEOGFROMGEOJSON('%s'))
  AND year = %s
''' % (json.dumps(test_region_geo.__geo_interface__), test_year))
print('Stations in region %s "%s" in %s:' % (test_region_id, test_region_name, test_year))
for row in resp:
    print('site %s (%s, %s)' % (row['site_num'], row['address'], row['cbsa_name']))

Stations in region 27980 "Kahului-Wailuku, HI" in 2007:
site 9001 (Haleakala National Park, Kahului-Wailuku-Lahaina, HI)
site 0006 (KAIHOI ST AND KAIOLOHIA ST, Kahului-Wailuku-Lahaina, HI)
site 9000 (Haleakala National Park, HI 96768, Kahului-Wailuku-Lahaina, HI)


As demonstrated in the proof of concept above, we can easily query for geographic traits in BigQuery. Let's load the ACS geometries into BigQuery directly, since once we compile the regions we can avoid handling massive geometry data.

In [12]:
import csv

'''
A utility function to upload a year of boundaries to BigQuery
'''
dataset_ref = bq.dataset(BQ_DATASET_ID)
def upload_year_to_bq(year):
    # convert to a nice table format
    table_data_path = '../_data/tmp/acs_cbsa_%s/bq.csv' % year
    print('%s: writing data to "%s"' % (year, table_data_path))
    if os.path.exists(table_data_path):
        os.remove(table_data_path)
    geo = load_geojson(year)
    with open(table_data_path, 'w') as csvfile:
        w = csv.DictWriter(csvfile, fieldnames=['geoid', 'name', 'geometry'])
        w.writeheader()
        for i in range(0, len(geo)):
            region = geo.loc[i]
            geoid = region['CBSAFP']
            name = region['NAME']
            geometry = region.geometry.__geo_interface__
            w.writerow({'geoid': geoid, 'name': name, 'geometry': json.dumps(geometry)})

    # move to bigquery
    table_ref = dataset_ref.table('acs_cbsa_boundaries_%s' % year)
    print('%s: setting up table "%s"' % (year, table_ref))
    bq.delete_table(table_ref, not_found_ok=True)
    bq.create_table(table_ref)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect = True
    with open(table_data_path, "rb") as source_file:
        print('%s: uploading...' % year)
        job = bq.load_table_from_file(source_file, table_ref, job_config=job_config)
        print('%s: done' % year)
    try:
        job.result() # wait for upload to complete
    except:
        for err in job.errors:
            print(err)
    return table_ref

In [13]:
# trial the function on one year
table_ref = upload_year_to_bq(get_acs_year(0))

2007: writing data to "../_data/tmp/acs_cbsa_2007/bq.csv"
=> loading 2007
2007: setting up table "TableReference(DatasetReference('eosc410-project', 'data'), 'acs_cbsa_boundaries_2007')"
2007: uploading...
2007: done


In [ ]:
# upload all years
# TODO